In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

### Load the experiment and corresponding libraries

In [2]:
project_path = os.path.join(Path.home(), "projects")
sys.path.append(project_path)  # enable import of "bias_transfer_recipes"
os.chdir(os.path.join(project_path,"bias_transfer_recipes/work/"))

In [3]:
from bias_transfer_recipes.bias_transfer_recipes.main import load_experiment
experiment = load_experiment(recipe="_2021_01_28_neurips", 
                             experiment="rep_matching", 
                             schema="bias_transfer_test",  # schema name will automatically be generated from recipe and experiment names 
                             base_dir=os.path.join(project_path,"bias_transfer_recipes/bias_transfer_recipes/"),
                             import_prefix="bias_transfer_recipes.bias_transfer_recipes.", 
                             dev_mode=True  
                            )

Connecting anix@datajoint-db.mlcloud.uni-tuebingen.de:3306


In [4]:
from bias_transfer.tables.transfer import *
from bias_transfer.tables.nnfabrik import *
schema

Connecting anix@datajoint-db.mlcloud.uni-tuebingen.de:3306


Schema `bias_transfer_test`

### Run experiments

In [5]:
from bias_transfer_recipes.bias_transfer_recipes.main import fill_tables, run_all_experiments

In [6]:
fill_tables(experiment.transfer_experiments)  

In [7]:
run_all_experiments(experiment.transfer_experiments)

  0%|          | 0/7 [00:00<?, ?it/s]

Loading dataset: TinyImageNet
Images already downloaded in ./data/image_classification/TinyImageNet
Model with 46028808 parameters.
==> Starting model Training 
Validation Epoch 0: 100%|██████████| 79/79 [00:15<00:00,  5.20it/s, RepresentationMatching loss =0.0096, RepresentationMonitor magnitude =1.0856, img_classification accuracy =1.8800, img_classification loss =5.1121, patience =0.0000]
Restoring best model from epoch! 1
Test Epoch 1: 100%|██████████| 79/79 [00:06<00:00, 11.80it/s, RepresentationMatching loss =0.0000, RepresentationMonitor magnitude =0.0000, img_classification accuracy =1.6100, img_classification loss =5.1139]


 14%|█▍        | 1/7 [08:19<49:56, 499.34s/it]

Nothing to delete
Deleting intermediate checkpoints...
Loading dataset: TinyImageNet
Images already downloaded in ./data/image_classification/TinyImageNet
Model with 46028808 parameters.
==> Starting model Training 
Validation Epoch 0: 100%|██████████| 79/79 [00:09<00:00,  8.48it/s, RepresentationMonitor magnitude =2.9166, img_classification accuracy =0.8600, img_classification loss =5.2114, patience =0.0000]
Restoring best model from epoch! 1
Test Epoch 1: 100%|██████████| 79/79 [00:06<00:00, 11.60it/s, RepresentationMonitor magnitude =0.0000, img_classification accuracy =1.1000, img_classification loss =5.1997]


 29%|██▊       | 2/7 [13:49<37:22, 448.49s/it]

Nothing to delete
Deleting intermediate checkpoints...
Loading dataset: TinyImageNet
Images already downloaded in ./data/image_classification/TinyImageNet
Model with 46028808 parameters.
==> Starting model Training 
Validation Epoch 0: 100%|██████████| 79/79 [00:09<00:00,  8.11it/s, RepresentationMonitor magnitude =2.9166, img_classification accuracy =0.8600, img_classification loss =5.2114, patience =0.0000]
Restoring best model from epoch! 1
Test Epoch 1: 100%|██████████| 79/79 [00:06<00:00, 11.63it/s, RepresentationMonitor magnitude =0.0000, img_classification accuracy =1.1000, img_classification loss =5.1997]


 43%|████▎     | 3/7 [19:20<27:33, 413.35s/it]

Nothing to delete
Deleting intermediate checkpoints...
Loading dataset: TinyImageNet
Images already downloaded in ./data/image_classification/TinyImageNet
Model with 46028808 parameters.
==> Starting model Training 
Training Epoch 1:  81%|████████▏ | 573/704 [02:23<00:32,  3.99it/s, img_classification accuracy =0.6613, img_classification loss =5.2881]

 43%|████▎     | 3/7 [21:53<29:11, 437.86s/it]

KeyboardInterrupt: 

In [57]:
%debug

> /usr/local/lib/python3.8/dist-packages/torch/nn/functional.py(2680)cosine_embedding_loss()
   2678     else:
   2679         reduction_enum = _Reduction.get_enum(reduction)
-> 2680     return torch.cosine_embedding_loss(input1, input2, target, margin, reduction_enum)
   2681 
   2682 



ipdb>  input1.shape


torch.Size([128, 256, 16, 16])


ipdb>  target.shape


torch.Size([128])


ipdb>  q


In [62]:
# dElete all jobs in error state:
if schema.database != "nnfabrik_core": 
    (schema.jobs & "status='error'").delete()

In [48]:
# drop the entire schema: THIS IS WILL DELETE EVERYTHING!
if schema.database != "nnfabrik_core": 
    schema.drop()

Proceed to delete entire schema `bias_transfer_test`? [yes, No]:  yes


In [46]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torch

experiment = "noisy_test"
run = "next run"
state = torch.load(Checkpoint().fetch("state",as_dict=True)[0]["state"])
tracker = state["tracker"]


In [47]:
def write_to_tensor_board(experiment, run, tracker):
    writer = SummaryWriter()
    def write_objective(obj_names, obj):
        if isinstance(obj,dict):
            for obj_name, sub_obj in obj.items():
                if obj_name == "normalization":
                    continue
                write_objective(obj_names+[obj_name],sub_obj)
        else:
            obj_val = tracker.get_objective(*obj_names)
            for i, val in enumerate(obj_val):
                writer.add_scalar('/'.join([run]+obj_names), val, i)
            
    write_objective([],tracker.objectives)
write_to_tensor_board(experiment, run, tracker)

/src/ml-utils/mlutils/tracking.py:144: RuntimeWarning: invalid value encountered in true_divide
  n_log[key] = l / norm


In [33]:
tracker.objectives

{'LR': 0,
 'Training': {'img_classification': {'loss': 0,
   'accuracy': 0,
   'normalization': 0},
  'NoiseAdvTraining': {'loss': 0, 'normalization': 0}},
 'Validation': {'img_classification': {'loss': 0,
   'accuracy': 0,
   'normalization': 0},
  'NoiseAdvTraining': {'loss': 0, 'normalization': 0}},
 'Test': {'NoiseAdvTraining': {'loss': 0, 'normalization': 0}}}

In [42]:
tracker.log

{'LR': [0.0003, 0.0003, 0.0003, 0],
 'Training': {'img_classification': {'loss': [312.89678955078125,
    477.7629699707031,
    316.83843994140625,
    0],
   'accuracy': [1900, 1900, 1100, 0],
   'normalization': [128, 128, 128, 0]},
  'NoiseAdvTraining': {'loss': [3.6585111916065216,
    5.0233617424964905,
    1.6073499619960785,
    0],
   'normalization': [60, 60, 60, 0]}},
 'Validation': {'img_classification': {'loss': [297.705322265625,
    298.799072265625,
    294.47955322265625,
    292.99310302734375],
   'accuracy': [2500, 1200, 3400, 2800],
   'normalization': [128, 128, 128, 128]},
  'NoiseAdvTraining': {'loss': [1.491750106215477,
    1.491749994456768,
    1.491749882698059,
    1.491750106215477],
   'normalization': [60, 60, 60, 60]}},
 'Test': {'NoiseAdvTraining': {'loss': [0, 0, 0, 0],
   'normalization': [0, 0, 0, 0]}}}

In [36]:
tracker.get_objective("LR")
tracker.get_objective("Training","img_classification","accuracy")

array([14.84375, 14.84375,  8.59375,      nan])

In [5]:
import torch
import numpy as np

In [6]:
from torchvision.datasets import SVHN
from torchvision import transforms

In [7]:
transform = transforms.Compose([
        transforms.ToTensor(),
#         transforms.Normalize((0.1307,), (0.3081,))
        ])
train = SVHN(
            root="/work/",
            split="train",
            download=True,
            transform=transform,
        )

Using downloaded and verified file: /work/train_32x32.mat


In [8]:
train_set = torch.utils.data.DataLoader(
        train,
        batch_size=64,
        shuffle=False,
    )
next(iter(train_set))[0].shape

torch.Size([64, 3, 32, 32])

In [65]:
mean = 0.
std = 0.
nb_samples = 0.
data_ = []
for data, _ in train_set:
    data_.append(data)
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples
dataset = torch.cat(data_)

In [66]:
print(mean,std)

tensor([0.4377, 0.4438, 0.4728]) tensor([0.1201, 0.1231, 0.1052])


In [67]:
std_ = dataset.std([0, 2, 3])
mean_ = dataset.mean([0, 2, 3])

In [68]:
std_

tensor([0.1980, 0.2010, 0.1970])

In [69]:
mean_

tensor([0.4377, 0.4438, 0.4728])

In [70]:
dataset.shape

torch.Size([73257, 3, 32, 32])

In [5]:
import datajoint as dj

In [7]:
x=list(range(10000))

In [8]:
import pickle as pkl

In [11]:
with open("x_out.pkl", "wb") as f:
    pkl.dump(x,f)

In [14]:
@schema
class DataStorage(dj.Manual):
    storage = "minio"
    
    @property
    def definition(self):
        definition = """
        # Contains the data generated by the transfer step, stored externally.
        id: varchar(128)
        ---
        data:            attach@{storage}
        """.format(
            storage=self.storage
        )
        return definition

In [15]:
DataStorage()

id,data


In [16]:
DataStorage.insert1({"id":"test","data":"./x_out.pkl"})

In [18]:
DataStorage()

id,data
test,=BLOB=


In [22]:
out = DataStorage.fetch1()

In [23]:
out

{'id': 'test', 'data': 'x_out.pkl'}